In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchsummary import summary

!pip install torchsummary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

You should consider upgrading via the 'D:\Python\envs\pytorch\python.exe -m pip install --upgrade pip' command.
cuda


In [2]:

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3) #input - 28*28*1, Channels - 32, Output - 28*28*32 Parameters - 1*1*32*32 + 32
        self.bn1 = nn.BatchNorm2d(32)
        self.dpt1 = nn.Dropout(0.25)  # Dropout - 25%

        self.conv2 = nn.Conv2d(32, 64, 3)  # Input - 26*26*32, Channels - 64, Output - 24*24*64 - Para - 9*32*64 + 64
        self.dpt2 = nn.Dropout(0.25)  # Dropout - 25%
        self.bn2 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(2, 2)  # Maxpool2D # Input - 24*24*64, Channels - 64, Output - 12*12*64 

        self.conv3 = nn.Conv2d(64, 32, 3)  # Input - 12*12*64, Channels - 64, Output - 10*10*32 - Para - 9*64*32 + 32
        self.dpt3 = nn.Dropout(0.25)  # Dropout - 25%
        self.bn3 = nn.BatchNorm2d(32)
        
        self.conv4 = nn.Conv2d(32, 64, 3)  # input -10*10*32 OUtput - 8*8*64 - Para - 9*32*64 + 64
        self.dpt4 = nn.Dropout(0.25)  # Dropout - 25%
        self.bn4 = nn.BatchNorm2d(64)
        
        self.conv5 = nn.Conv2d(64, 32, 3)  # input -8*8*64 OUtput - 6*6*32 - Para - 9*32*64 + 32
        self.dpt5 = nn.Dropout(0.25)  # Dropout - 25%
        self.bn5 = nn.BatchNorm2d(32)
        self.pool2 = nn.MaxPool2d(2, 2)  # input -6*6*32 OUtput -3*3*32 - Para - 9*32*32
        

        self.conv6 = nn.Conv2d(32, 32, 1)  # input -3*3*32 OUtput - 3*3*32 - Para - 1*32*32 +32
        self.conv7 = nn.Conv2d(32, 10, 3)  # input -3*3*32 OUtput - 1*1*10 - Para - 9*32*10 + 10


    def forward(self,x):
        x = self.dpt1(F.relu(self.bn1(self.conv1(x))))
        x = self.dpt2(self.pool1(self.bn2(F.relu(self.conv2(x)))))
        x = self.dpt3(self.bn3(F.relu(self.conv3(x))))
        x = self.dpt4(self.bn4(F.relu(self.conv4(x))))
        x = self.pool2(self.dpt5(self.bn5(F.relu(self.conv5(x)))))
        x = F.relu(self.conv6(x))
        x = self.conv7(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [3]:

model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 26, 26]             320
       BatchNorm2d-2           [-1, 32, 26, 26]              64
           Dropout-3           [-1, 32, 26, 26]               0
            Conv2d-4           [-1, 64, 24, 24]          18,496
       BatchNorm2d-5           [-1, 64, 24, 24]             128
         MaxPool2d-6           [-1, 64, 12, 12]               0
           Dropout-7           [-1, 64, 12, 12]               0
            Conv2d-8           [-1, 32, 10, 10]          18,464
       BatchNorm2d-9           [-1, 32, 10, 10]              64
          Dropout-10           [-1, 32, 10, 10]               0
           Conv2d-11             [-1, 64, 8, 8]          18,496
      BatchNorm2d-12             [-1, 64, 8, 8]             128
          Dropout-13             [-1, 64, 8, 8]               0
           Conv2d-14             [-1, 3

In [7]:
torch.manual_seed(1)
batch_size = 256

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [5]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n' .format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [8]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(15):
    print('EPOCH: ', epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/235 [00:00<?, ?it/s]
Test set: Average loss: 0.0627, Accuracy: 9831/10000 (98%)

EPOCH:  1
  0%|          | 0/235 [00:00<?, ?it/s]
Test set: Average loss: 0.0481, Accuracy: 9860/10000 (99%)

EPOCH:  2
  0%|          | 0/235 [00:00<?, ?it/s]
Test set: Average loss: 0.0373, Accuracy: 9886/10000 (99%)

EPOCH:  3
  0%|          | 0/235 [00:00<?, ?it/s]
Test set: Average loss: 0.0333, Accuracy: 9900/10000 (99%)

EPOCH:  4
  0%|          | 0/235 [00:00<?, ?it/s]
Test set: Average loss: 0.0303, Accuracy: 9909/10000 (99%)

EPOCH:  5
  0%|          | 0/235 [00:00<?, ?it/s]
Test set: Average loss: 0.0271, Accuracy: 9912/10000 (99%)

EPOCH:  6
  0%|          | 0/235 [00:00<?, ?it/s]
Test set: Average loss: 0.0248, Accuracy: 9922/10000 (99%)

EPOCH:  7
  0%|          | 0/235 [00:00<?, ?it/s]
Test set: Average loss: 0.0231, Accuracy: 9931/10000 (99%)

EPOCH:  8
  0%|          | 0/235 [00:00<?, ?it/s]
Test set: Average loss: 0.0229, Accuracy: 9927/10000 (99%)

EPOCH:  9
  0%|     